In [2]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.enable("json")

DataTransformerRegistry.enable('json')

# User inputs

In [5]:
start_date = '20221130'
end_date = '20230509'

tidy_dataset_fn = f"../sos/tidy_df_30Min_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_dataset_5min_fn = f"../sos/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_daily_dataset_output_fn = f"tidy_df_daily_{start_date}_{end_date}_noplanar_fit.parquet"

# Load data

In [6]:
try:
    tidy_df_30Min = pd.read_parquet(
        tidy_dataset_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])

try:
    tidy_df_5Min = pd.read_parquet(
        tidy_dataset_5min_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])

# Examine pot. virtual temperature gradient

In [7]:
np.abs(tidy_df_30Min[tidy_df_30Min.measurement == 'temperature gradient'].value).min()

1.4920204573882438e-06

In [8]:
alt.Chart(
    tidy_df_30Min.query("measurement == 'temperature gradient'").query("height < 5")
).mark_line().encode(
    x = 'time:T',
    y = 'value:Q',
    column='height:O'
).properties(width=200, height = 200) & alt.Chart(
    tidy_df_30Min.query("measurement == 'temperature gradient'").query("height < 5")
).mark_bar().encode(
    alt.X('value:Q').bin(step=0.1),
    alt.Y("count():Q"),    
    alt.Column('height:O')
).properties(width=200, height = 200)

alt.VConcatChart(...)

In [9]:
src = tidy_df_30Min.query("variable == 'temp_gradient_3m_c'")
neutral_times = src[src['value'].abs() < 0.01].time

In [10]:
alt.Chart(
    tidy_df_30Min[
        tidy_df_30Min.time.isin(neutral_times.sample(16))
    ].query("measurement == 'wind speed'").query("tower == 'c'")
).mark_line().encode(
    alt.X("value:Q").title("Wind speed (m/s)").sort('-y'),
    alt.Y("height:Q").title("Height (m)"),
    alt.Facet("time:O", columns=8)
).properties(width = 125, height = 125)

alt.Chart(...)

In [11]:
src = tidy_df_30Min[
    tidy_df_30Min.time.isin(neutral_times)
].query("tower == 'c'")
src_snowdepth = tidy_df_30Min[
    tidy_df_30Min.measurement == 'snow depth'
]
src_snowdepth = src_snowdepth[['time', 'value']].set_index('time').rename(columns={'value': 'snow_depth'})
src = src[src.measurement.isin([
    'wind speed',
    'shear velocity',
    'snow depth'
])]
src = src[~src.variable.str.contains("predicted")]
src = src.pivot_table(index=['time'], values='value', columns='variable')
src = src.join(src_snowdepth)
src.head()

,spd_10m_c,spd_15m_c,spd_20m_c,spd_2m_c,spd_3m_c,spd_5m_c,u*_10m_c,u*_15m_c,u*_20m_c,u*_2m_c,u*_3m_c,u*_5m_c,snow_depth
time,,,,,,,,,,,,,
2022-12-02 10:30:00,5.852733,5.990304,6.163234,5.169695,5.503640,5.802035,0.771105,0.711912,0.749025,0.772614,0.748710,0.600726,0.346144
2022-12-02 11:00:00,5.108618,5.132134,5.137251,4.701602,4.848611,5.051483,0.792359,0.733981,0.720212,0.809777,0.727379,0.728357,0.346144
2022-12-02 11:30:00,4.945608,5.086113,5.158172,4.254836,4.424224,4.764127,0.827612,0.762404,0.772660,0.802487,0.794172,0.635123,0.346144
2022-12-02 12:00:00,6.954772,7.209726,7.195507,5.955409,6.454049,6.821709,0.922849,1.007616,1.199078,0.840452,0.820263,0.595426,0.346144
2022-12-02 12:30:00,4.535348,4.960222,4.781742,4.116060,4.270485,4.554066,0.813880,0.840683,0.887759,0.753841,0.708530,0.631443,0.346144


# Filter out data without monotonically increasing wind speeds 

In [12]:
def monotonically_increasing(l):
    return all(x < y for x, y in zip(l, l[1:]))

src['is_monotonic_increasing'] = src.apply(
    lambda row: monotonically_increasing([
            row['spd_2m_c'], 
            row['spd_3m_c'], 
            row['spd_5m_c'], 
            row['spd_10m_c'], 
            row['spd_15m_c'], 
            row['spd_20m_c']
    ]),
    axis = 1
)

In [13]:
src = src[src.is_monotonic_increasing]

# Solve for $z_0$ assuming $d = 0$

https://www.eol.ucar.edu/content/calculation-roughness-length-and-displacement-height

In [14]:
von_karman = 0.4

In [15]:
d = 0.0
src['z0_2m_c'] = (2 - d - src['snow_depth'])/np.exp(src['spd_2m_c']*von_karman/src['u*_2m_c'])
src['z0_3m_c'] = (3 - d - src['snow_depth'])/np.exp(src['spd_3m_c']*von_karman/src['u*_3m_c'])
src['z0_5m_c'] = (5 - d - src['snow_depth'])/np.exp(src['spd_5m_c']*von_karman/src['u*_5m_c'])
src['z0_10m_c'] = (10 - d - src['snow_depth'])/np.exp(src['spd_10m_c']*von_karman/src['u*_10m_c'])
src['z0_15m_c'] = (15 - d - src['snow_depth'])/np.exp(src['spd_15m_c']*von_karman/src['u*_15m_c'])
src['z0_20m_c'] = (20 - d - src['snow_depth'])/np.exp(src['spd_20m_c']*von_karman/src['u*_20m_c'])

In [16]:
variables = [
        'z0_2m_c', 
        'z0_3m_c', 
        'z0_5m_c', 
        'z0_10m_c', 
        'z0_15m_c', 
        'z0_20m_c', 
    ]
alt.Chart(
    src[variables].reset_index()
).transform_fold(
    variables
).transform_filter(
    alt.FieldOneOfPredicate('key', ['z0_2m_c', 'z0_3m_c'])
).mark_circle().encode(
    alt.X('time:T'),
    alt.Y("value:Q").scale(type='log'),
    alt.Row("key:N", sort=variables)
).properties(height = 100, width = 400)

alt.Chart(...)

In [19]:
alt.Chart(
    src[['z0_3m_c']].resample("1D").median().reset_index()
).mark_circle(size=50).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
) + alt.Chart(
    src[['z0_3m_c']].resample("1D").median().reset_index()
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
).properties(height = 100)

alt.LayerChart(...)

In [14]:
alt.Chart(
    src[['z0_3m_c']].resample("W-MON").median().reset_index()
).mark_circle(size=100).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
) + alt.Chart(
    src[['z0_3m_c']].resample("W-MON").median().reset_index()
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
).properties(height = 100)

alt.LayerChart(...)

In [15]:
src[['z0_3m_c']].resample("W-MON").median().reset_index()

,time,z0_3m_c
0,2022-12-05,0.177667
1,2022-12-12,0.007912
2,2022-12-19,0.005446
3,2022-12-26,0.052083
4,2023-01-02,0.000998
5,2023-01-09,0.007609
6,2023-01-16,0.001780
7,2023-01-23,0.001265
8,2023-01-30,0.004634
9,2023-02-06,0.002709


# Solve for $z_0$ by fitting a line

In [16]:
y = src.iloc[0][
    ['spd_2m_c', 'spd_3m_c', 'spd_5m_c', 'spd_10m_c', 'spd_15m_c', 'spd_20m_c']
].values
x = np.array([np.log(2), np.log(3), np.log(5), np.log(10), np.log(15), np.log(20)])

In [17]:
import matplotlib.pyplot as plt

In [18]:
import scipy

In [19]:
regress_result = scipy.stats.linregress(x[:-1],y[:-1])
regress_result

AttributeError: 'float' object has no attribute 'shape'

In [ ]:
xs = np.linspace(0.5, 3.5, 10)
ys = regress_result.slope*xs + regress_result.intercept

plt.scatter(x,y)
plt.plot(xs,ys)

In [ ]:
u_star = regress_result.slope*von_karman
u_star

In [ ]:
z0 = np.exp(-regress_result.intercept * von_karman / u_star)
z0

# Use Andreas et al. 2010 Method

In [ ]:
measurements = tidy_df_30Min.set_index("time").sort_index().loc['2022-11-30': '2023-05-9']

$ u_* = \sqrt{\overline{u'w'}}$

In [ ]:
shear_velocity = (
    measurements.query("variable == 'u_w_rot__3m_c'")['value']
)**2

$L = \dfrac{\overline{\theta_v}}{k g}\dfrac{u_*^3}{\overline{w\theta_v}}$

where

$\overline{\Theta_v} = \text{average virtual temperature}$

$\overline{w \theta_v} = \text{flux of virtual temperature}$

In [ ]:
k = 0.4
g = 9.81 # m/s^2

obukhov_length = (
    measurements.query("variable == 'Tvirtual_3m_c'")['value'] / (k*g)
) * (
    shear_velocity**3 / measurements.query("variable == 'w_tc__3m_c'")['value']
)

In [ ]:
beta_g = 1.25
z_i = 600 #meters, assumed constant depth of the convective boundary layer - may not apply for our environment

w_star = shear_velocity*(
    - z_i / ( k * obukhov_length )
)**(1/3)

#unstable
s_r__unstable = (
    (
        measurements.query("variable == 'spd_3m_c'")['value']**2
    ) + (beta_g**2 * w_star**2)
)**0.5

s_r__stable = (
    measurements.query("variable == 'spd_3m_c'")['value'] + 0.5*(1 / np.cosh(
        measurements.query("variable == 'spd_3m_c'")['value']
    ))
)

$ C_{Dr} = \dfrac{u_*^2}{S_r^2} $

In [ ]:
C_Dr_unstable = shear_velocity**2 / s_r__unstable**2
C_Dr_stable = shear_velocity**2 / s_r__stable**2

In [ ]:
# obukhov_length
gamma = 16
z = 3

x_unstable = (1 - gamma*(z/obukhov_length))**0.25
phi_unstable = 2 * np.log((1 + x_unstable)/2) + np.log((1 + x_unstable**2)/2) - 2*np.arctan(x_unstable) + np.pi / 2

In [ ]:
a_m = 5
b_m = a_m/6.5
B_m = ((1 - b_m)/b_m)**(1/3)

In [ ]:
x_stable = (1 + z / obukhov_length)**(1/3)
phi_stable = (-3*a_m/b_m) * ( x - 1) + (a_m*B_m/(2*b_m))*(
    2*np.log(    (x_stable + B_m) / (z + B_m)   )
     - np.log(
            (x_stable**2 - x*B_m + B_m**2) / (1 - B_m + B_m**2)
     )
     + 2*np.sqrt(3) * (
         np.arctan(((2*x) - B_m) / (np.sqrt(3)*B_m)) - np.arctan( (2 - B_m) / (np.sqrt(3)*B_m))
        )
)

In [ ]:
df = pd.DataFrame({
    'L' : obukhov_length,
    'C_Dr_stable': C_Dr_stable,
    'C_Dr_unstable': C_Dr_unstable,
    'phi_m_stable': phi_stable,
    'phi_m_unstable': phi_unstable,
})

In [ ]:
# if L < 0, unstable

df['C_Dr'] = df.apply(
    lambda row: row['C_Dr_unstable'] if row['L'] < 0 else row['C_Dr_stable'],
    axis=1
)
df['phi'] = df.apply(
    lambda row: row['phi_m_unstable'] if row['L'] < 0 else row['phi_m_stable'],
    axis=1
)

In [ ]:
df

$ z_0 = r \exp{\Big(
    - ( k*C_{dr}^{(-1/2)} + \psi_m(r/L))    
\Big)} $

In [ ]:
df['z0'] = z * np.exp(
    - (k * df['C_Dr']**(-0.5) + df['phi'])
)